In [1]:
import torch

import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset, Dataset

from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# from sklearn.model_selection import train_test_split

from torchvision.transforms import transforms

from sklearn.neighbors import NearestNeighbors

import numpy as np

# def get_augmentations():

#     return transforms.Compose([transforms.RandomHorizontalFlip(),

#                                transforms.RandomVerticalFlip(),

#                                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),

#                                ])

def generate_synthetic(X, labels, n_neighbors=3):

    X = X.copy()

    print(X.shape)

    X_where_y0 = X[labels == 0]  # majority class

    X_where_y1 = X[labels == 1]

    X_where_y2 = X[labels == 2]

    y0_num = X_where_y0.shape[0]

    y1_num = X_where_y1.shape[0]

    y2_num = X_where_y2.shape[0]

    X_w_y1_reshaped = X_where_y1.reshape(X_where_y1.shape[0], -1)

    X_w_y2_reshaped = X_where_y2.reshape(X_where_y2.shape[0], -1)

    y1_upsample = y0_num - y1_num

    y2_upsample = y0_num - y2_num

    X_w_y1_synthetic = smote(X_w_y1_reshaped, y1_upsample, n_neighbors)

    X_w_y2_synthetic = smote(X_w_y2_reshaped, y2_upsample, n_neighbors)

    X_w_y1_synthetic = X_w_y1_synthetic.reshape(-1, *X_where_y1.shape[1:])

    X_w_y2_synthetic = X_w_y2_synthetic.reshape(-1, *X_where_y2.shape[1:])

    X_oversampled = np.vstack([X, X_w_y1_synthetic, X_w_y2_synthetic])

    y_oversampled = np.hstack([

        labels,

        np.ones(X_w_y1_synthetic.shape[0]),

        np.full(X_w_y2_synthetic.shape[0], 2)

    ])

    return X_oversampled, y_oversampled

def smote(X, num_oversamples, n_neighbors=5):

    n_samples, n_features = X.shape

    synthetic_samples = np.zeros((num_oversamples, n_features))

    nn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree')

    nn.fit(X)

    indices = np.random.randint(0, n_samples, size=num_oversamples)

    samples = X[indices]

    nnres = nn.kneighbors(samples, return_distance=False)

    nn_indices = nnres[np.arange(num_oversamples), np.random.randint(0, n_neighbors, size=num_oversamples)]

    nn_samples = X[nn_indices]

    diffs = nn_samples - samples

    synthetic_samples = samples + diffs * np.random.random(size=(num_oversamples, 1))

    return synthetic_samples.reshape(num_oversamples, *X.shape[1:])

def drop_nan_y(X, y):

    nan_indices = np.argwhere(np.isnan(y)).squeeze()

    mask = np.ones(y.shape, bool)

    mask[nan_indices] = False

    X = X[mask]

    y = y[mask]

    return X, y

def clean_x_data(X):

    X[np.isnan(X)] = np.nanmedian(X)

    X[X < 0] = 0

    X[X > 255] = 255

    # lower = np.percentile(X, 25) * 1.15

    # upper = np.percentile(X, 75) * 1.5

    # X[X < lower] = lower

    # X[X > upper] = upper

    return X

# class CustomNeuralNetwork(nn.Module):

#     def __init__(self, input_size, classes=3, drop_prob=0.5):

#         super().__init__()

#         self.network = nn.Sequential(

#             nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),

#             nn.ReLU(),

#             nn.MaxPool2d(kernel_size=2, stride=2),

#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),

#             nn.ReLU(),

#             nn.MaxPool2d(kernel_size=2, stride=2),

#             nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),

#             nn.ReLU(),

#             nn.MaxPool2d(kernel_size=2, stride=2),

#             nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3),

#             nn.MaxPool2d(kernel_size=2, stride=2),

#             nn.Flatten(),

#         )

#

#         self.fc = nn.Sequential(

#             nn.Linear(256, 128),

#             nn.ReLU(),

#             nn.Linear(128, 64),  # New fully connected layer

#             nn.ReLU(),

#             nn.Linear(64, 32),

#             nn.ReLU(),

#             nn.Linear(32, classes)

#         )

#

#

#     def forward(self, x):

#         x = self.network(x)

#         # print(x.shape)

#         x = self.fc(x)

#         return x

class CustomNeuralNetwork(nn.Module):

    def __init__(self, input_size, classes=3, drop_prob=0.3):

        super().__init__()

        self.network = nn.Sequential(

            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),

            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.ReLU(),

            nn.Dropout(drop_prob),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),

            nn.ReLU(),

            # nn.Dropout(drop_prob),

            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),

        )

        self.fc = nn.Sequential(

            nn.Linear(256, 64),

            nn.ReLU(),

            nn.Linear(64, 32),

            nn.ReLU(),

            nn.Linear(32, classes)

        )

    def forward(self, x):

        x = self.network(x)

        # print(x.shape)

        x = self.fc(x)

        return x

class Model:

    """

    This class represents an AI model.

    """

    def __init__(self,

                 batch_size=10,

                 epochs=10,  # epochs seem to get worse after about 10 at num_components=256

                 # learning_rate=1e-3,

                 criterion=nn.CrossEntropyLoss,

                 num_components=256,

                 scaler=MinMaxScaler(),

                 learning_rate=1e-3,

                 ):

        """

        Constructor for Model class.

        Parameters

        ----------

        self : object

            The instance of the object passed by Python.

        """

        # TODO: Replace the following code with your own initialization code.

        # self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # self.device = torch.device("cpu")

        self.optimizer = None

        self.model = None

        self.batch_size = batch_size

        self.epochs = epochs

        self.learning_rate = learning_rate

        self.criterion = criterion()

        self.num_components = num_components

        self.pca = PCA(n_components=num_components, svd_solver='full')

        self.scaler = scaler

    def fit(self, X, y):

        """

        Train the model using the input data.

        Parameters

        ----------

        X : ndarray of shape (n_samples, channel, height, width)

            Training data.

        y : ndarray of shape (n_samples,)

            Target values.

        Returns

        -------

        self : object

            Returns an instance of the trained model.

        """

        # TODO: Add your training code.

        self.model = CustomNeuralNetwork(input_size=self.num_components)

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        print('start')

        X, y = drop_nan_y(X, y)

        X = clean_x_data(X)

        # print("pre-synthetic")

        X, y = generate_synthetic(X, y, 5)

        # print(y.min())

        # X, X_test, y, y_test = train_test_split(X, y, test_size=100)

        # print(y.min())

        # Flatten and normalize the data

        flattened_data = X.reshape(X.shape[0], -1)

        normalized_data = self.scaler.fit_transform(flattened_data)

        # print("pre-pca")

        # print(y.min())

        pca_result = self.pca.fit_transform(normalized_data)

        reconstructed = self.pca.inverse_transform(pca_result)

        original_pca = reconstructed.reshape(-1, *X.shape[1:])

        pca_result_tensor = torch.tensor(original_pca, dtype=torch.float32)  #.to(self.device)

        labels_tensor = torch.tensor(y, dtype=torch.long)  # .to(self.device)

        # print(y.min())

        # dataset = CustomTensorDataset(tensors=(pca_result_tensor, labels_tensor), transform=get_augmentations())

        dataset = TensorDataset(pca_result_tensor, labels_tensor)

        train_loader = DataLoader(dataset=dataset, batch_size=self.batch_size, shuffle=True)

        # print("pre-epoch")

        epoch_losses = []

        for epoch in range(self.epochs):

            epoch_loss = 0

            # print(f"Epoch {epoch+1}")

            for inputs, labels in train_loader:

                # print(inputs, labels)

                self.optimizer.zero_grad()

                outputs = self.model(inputs)

                loss = self.criterion(outputs, labels)

                loss.backward()

                self.optimizer.step()

                epoch_loss += loss.item()

            epoch_losses.append(epoch_loss / len(train_loader))

            print(f"Epoch {epoch + 1} loss: {epoch_losses[-1]}")

        return self

    def predict(self, X):

        """

        Use the trained model to make predictions.

        Parameters

        ----------

        X : ndarray of shape (n_samples, channel, height, width)

            Input data.

        Returns

        -------

        ndarray of shape (n_samples,)

        Predicted target values per element in X.

        """

        # TODO: Replace the following code with your own prediction code.

        X = clean_x_data(X)

        X = torch.from_numpy(X).float()

        # X.to(self.device)

        self.model.eval()

        flattened_data = X.reshape(X.shape[0], -1)

        normalized_data = self.scaler.transform(flattened_data)

        pca_result = self.pca.transform(normalized_data)

        reconstructed = self.pca.inverse_transform(pca_result)

        original_pca = reconstructed.reshape(-1, *X.shape[1:])

        print("fit shape:", pca_result.shape)

        original_pca = torch.tensor(original_pca, dtype=torch.float32)  #.to(self.device)

        with torch.no_grad():

            outputs = self.model(original_pca)

        return outputs.detach().numpy().argmax(axis=1)

In [2]:
%load_ext memory_profiler

In [3]:
%%time
%%memit

# N fold cross validation
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

with open('data.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True).item()
    X = data['image']
    y = data['label']


nan_indices = np.argwhere(np.isnan(y)).squeeze()
mask = np.ones(y.shape, bool)
mask[nan_indices] = False
X = X[mask]
y = y[mask]

num_folds = 10

model = Model()
kf = KFold(n_splits=num_folds, shuffle=True, random_state=2109)

f1_scores = []
i = 0

for train_index, test_index in kf.split(X):
    i += 1
    print("Fold:", i)
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]

    model.fit(X=X_train, y=y_train)

    predictions = model.predict(X_test)
    print("y_test values:", y_test[y_test==0].shape, y_test[y_test==1].shape, y_test[y_test==2].shape)
    print("predictions:", predictions[predictions==0].shape, predictions[predictions==1].shape, predictions[predictions==2].shape)

    score = f1_score(y_test, predictions, average='macro')

    f1_scores.append(score)
    print("f1:", score)

print("F1:", f1_scores)
print("Mean:", np.mean(f1_scores))
print("Std:", np.std(f1_scores))
print("Max:", np.max(f1_scores))
print("Min:", np.min(f1_scores))


Fold: 1
start
(2358, 3, 16, 16)
Epoch 1 loss: 0.41708080986310603
Epoch 2 loss: 0.16846878853494324
Epoch 3 loss: 0.12867764607753884
Epoch 4 loss: 0.0952498455428415
Epoch 5 loss: 0.07896771003108236
Epoch 6 loss: 0.07571414464089482
Epoch 7 loss: 0.05718140528039873
Epoch 8 loss: 0.05054400922858523
Epoch 9 loss: 0.05077423851944079
Epoch 10 loss: 0.04468724971675406
fit shape: (262, 256)
y_test values: (235,) (26,) (1,)
predictions: (230,) (29,) (3,)
f1: 0.7166829586184426
Fold: 2
start
(2358, 3, 16, 16)
Epoch 1 loss: 0.37655568522291244
Epoch 2 loss: 0.17109328936366075
Epoch 3 loss: 0.12864144107613204
Epoch 4 loss: 0.10938909662068931
Epoch 5 loss: 0.08754905897083473
Epoch 6 loss: 0.082910882219847
Epoch 7 loss: 0.06899491239625452
Epoch 8 loss: 0.062298821582082595
Epoch 9 loss: 0.05867836138243756
Epoch 10 loss: 0.04650756284907187
fit shape: (262, 256)
y_test values: (246,) (16,) (0,)
predictions: (232,) (28,) (2,)
f1: 0.48326359832635984
Fold: 3
start
(2358, 3, 16, 16)
Epoch